In [1]:
import os
import pandas as pd

adni_jihyun_resilience = pd.read_csv('C:/Users/BREIN/Desktop/copathology_visualization_temp/data/adni/ADNI4_resilience_inference.csv')
adni_nxt = pd.read_csv('C:/Users/BREIN/Desktop/copathology_visualization_temp/data/stage_data/ptau_volume_model/ADNI4_3.csv')

In [2]:
import pandas as pd
import numpy as np

# Example
prob_cols = ['pred_tau_stage_aa/low', 'pred_tau_stage_aa/mid', 'pred_tau_stage_aa/high']  # your probability columns

# Create a copy of just the probability columns
adni_nxt_bin = adni_nxt[prob_cols].copy()

# For each row, set the max to 1, others to 0
adni_nxt_bin[:] = (adni_nxt_bin.values == adni_nxt_bin.values.max(axis=1)[:, None]).astype(int)

# Merge back if you want
for col in prob_cols:
    adni_nxt[col + '_winner'] = adni_nxt_bin[col]

print(adni_nxt[prob_cols + [c + '_winner' for c in prob_cols]].head())


   pred_tau_stage_aa/low  pred_tau_stage_aa/mid  pred_tau_stage_aa/high  \
0               0.921026               0.044861                0.034113   
1               0.890819               0.078589                0.030592   
2               0.207066               0.479581                0.313353   
3               0.739776               0.187160                0.073064   
4               0.387115               0.351926                0.260959   

   pred_tau_stage_aa/low_winner  pred_tau_stage_aa/mid_winner  \
0                           1.0                           0.0   
1                           1.0                           0.0   
2                           0.0                           1.0   
3                           1.0                           0.0   
4                           1.0                           0.0   

   pred_tau_stage_aa/high_winner  
0                            0.0  
1                            0.0  
2                            0.0  
3                 

In [11]:
real_low = adni_nxt[adni_nxt['tau_stage_aa/low']==1]
real_mid = adni_nxt[adni_nxt['tau_stage_aa/mid']==1]
real_low_pred_high = real_low[real_low['pred_tau_stage_aa/high_winner']==1]
real_low_pred_mid = real_low[real_low['pred_tau_stage_aa/mid_winner']==1]
real_mid_pred_high = real_mid[real_mid['pred_tau_stage_aa/high_winner']==1]

# total_nxt_vuln_df = pd.concat([real_low_pred_high, real_low_pred_mid, real_mid_pred_high], ignore_index=True)
total_nxt_vuln_df = pd.concat([real_low_pred_high, real_low_pred_mid], ignore_index=True)

print(real_low_pred_high.shape)
print(real_low_pred_mid.shape)
print(real_mid_pred_high.shape)
print(total_nxt_vuln_df.shape)

total_nxt_vuln_df["FULL_ID"] = total_nxt_vuln_df["FULL_ID"].str.replace("_M", "_m", regex=False)
# total_nxt_vuln_df = total_nxt_vuln_df.merge(adni_jihyun_resilience, on='FULL_ID')

total_nxt_vuln_df =total_nxt_vuln_df.merge(
    adni_jihyun_resilience[["FULL_ID", "TN_group", "standardized_residual"]],
    on="FULL_ID",
    how="left"
)

(3, 576)
(3, 576)
(13, 576)
(6, 576)


In [10]:
total_nxt_vuln_df['TN_group'].value_counts()

TN_group
Vulnerable    12
Canonical      1
Name: count, dtype: int64

In [13]:
total_nxt_vuln_df.to_csv('C:/Users/BREIN/Desktop/copathology_visualization_temp/data/temp.csv', index=False)